In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time, lxml, requests, re
import pandas as pd
from gensim.summarization import summarize
from googletrans import Translator
from Phase1 import execute
from my1 import fd
from text3 import send_message

# 팝업 전담 일진 : 거지같은 팝업 제거 성공!!!!!! 안녕 지겨웠고 다신 보지 말자!
def kill_iframe(driver):
    iframe = driver.find_element_by_class_name('truste_popframe')
    # 이게 키포인트! iframe에 포커스하면 iframe 안의 엘리먼트를 찾을 수 있습니다!
    driver.switch_to.frame(iframe)
    target = driver.find_element_by_link_text('모두 거절')
    target.click()
    time.sleep(5)
    target = driver.find_element_by_link_text('닫기')
    target.click()

def main_process(driver):
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    #print(soup.prettify())

    #타이틀
    title = soup.find('h1', class_='title').get_text()
    title = re.sub('[\s]+', ' ', title)
    print(title.split(' '))
    category = title.split(' ')[0]
    count = 0
    select_lists = []
    print(category)
    swit = -1 # 대분류를 나타내는 변수

    # 현재 페이지가 클래스나 인터페이스인 경우(첫번째 대분류)
    if category == 'Class' or category == 'Interface':
        swit = 1 # 대분류 첫번째 표식
        titles = soup.findAll('div', class_='sub-title')

        # 모듈
        module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, module])
        
        # 패키지
        package = ['패키지', titles[1].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[1].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, package])
        
        # 본문
        content = soup.find('div', class_='block').get_text()

        # 상세 : 구현 인터페이스, 구현 클래스 등......
        # 구현한 인터페이스
        section = soup.find('section', class_='description')
        cd = False
        if section == None: 
            section = soup.find('section', id='class-description')
            cd = True
        if section != None:
            notes = section.findAll('dl', class_='notes')
            idx = -1
            print('section in')
            for note in notes:
                idx += 1
                dt = note.find('dt')
                print('dt : ', dt.get_text())
                if dt.get_text() == 'All Implemented Interfaces:':
                    print('1')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['구현한 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                    
                elif dt.get_text() == 'All Superinterfaces:':
                    print('2')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['슈퍼 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                
                elif dt.get_text() == 'All Known Implementing Classes:':
                    print('3')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['구현 클래스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                
                elif dt.get_text() == 'All Known Subinterfaces:': 
                    print('4')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['서브 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                    
                elif dt.get_text() == 'Enclosing interface:': 
                    print('5')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    print(imple_inters)
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['Enclosing 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                    
                # 서브 클래스
                elif dt.get_text() == 'Direct Known Subclasses:':
                    print('6')
                    known_sub_title = note.find('dt').get_text()
                    known_subs = note.select('dd > code > a')
                    known_subs = [a.get_text() for a in known_subs]
                    if cd == True: known_targets = driver.find_elements_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: known_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    known_sub = ['서브 클래스', [[name, target] for name, target in zip(known_subs, known_targets)]]
                    count += 1
                    select_lists.append([count, known_sub])

                
        sig_raw = soup.find('div', class_='type-signature').get_text()
        sig = re.sub('\n', ' ', sig_raw)

        # 중첩 클래스
        nested_raw = soup.find('section', id='nested.class.summary')
        if nested_raw != None:
            nest_title = nested_raw.find('h2').get_text()
            nest_table = nested_raw.find('div', class_='summary-table three-column-summary')
            if nest_table != None:
                nest_col1 = nest_table.findAll('div', class_='col-first')
                nest_col2 = nest_table.findAll('div', class_='col-second')
                nest_col3 = nest_table.findAll('div', class_='col-last')

                nest_col1 = [target.get_text() for target in nest_col1]
                nest_col2 = [target.get_text() for target in nest_col2]
                nest_col3 = [re.sub('\n', ' ', target.get_text()) for target in nest_col3]

                nest_target = driver.find_element_by_id('nested.class.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                nest_target = [scol.find_elements_by_tag_name('a') for scol in nest_target] #
                
                nest_df = pd.DataFrame([nest_col1[1:], nest_col2[1:], nest_col3[1:], nest_target[1:]]).T
                nest = ['중첩 클래스', nest_df] #######

                count += 1
                select_lists.append([count, nest])

        # 필드
        field_raw = soup.find('section', id='field.summary')
        if field_raw != None:
            field_title = field_raw.find('h2').get_text()
            field_table = field_raw.find('div', class_='summary-table three-column-summary')
            
            if field_table != None:
                field_col1 = field_table.findAll('div', class_='col-first')
                field_col2 = field_table.findAll('div', class_='col-second')
                field_col3 = field_table.findAll('div', class_='col-last')

                field_col1 = [target.get_text() for target in field_col1]
                field_col2 = [target.get_text() for target in field_col2]
                field_col3 = [re.sub('\n', ' ', target.get_text()) for target in field_col3]

                field_target = driver.find_element_by_id('field.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                field_target = [scol.find_elements_by_tag_name('a') for scol in field_target] #
                
                field_df = pd.DataFrame([field_col1[1:], field_col2[1:], field_col3[1:], field_target[1:]]).T
                field = ['필드', field_df] #######

                count += 1
                select_lists.append([count, field])

        # 생성자
        const_cols = 0
        const_hipen = False
        const_sel = -1 # 별거 없음. 자기 자신의 번호를 여기다 저장. 나중에 선택할 때 생성자를 선택했다는 식별용 -1은 생성자 없다는 뜻
        const_raw = soup.find('section', id='constructor.summary') 
        if const_raw == None: 
            const_hipen = True
            const_raw = soup.find('section', id='constructor-summary') 
        if const_raw != None:
            const_title = const_raw.find('h2').get_text()
            
            const_table = const_raw.find('div', class_='summary-table two-column-summary')
            if const_table == None: # 컬럼이 3개인 생성자 목록일 때
                const_table = const_raw.find('div', class_='summary-table three-column-summary')
                if const_table != None: 
                    const_cols = 3
                    const_first = const_table.findAll('div', class_='col-first')[1:]
                    const_name = const_table.findAll('div', class_='col-constructor-name')
                    const_last = const_table.findAll('div', class_='col-last')[1:]
                    
                    const_first = [target.get_text() for target in const_first]
                    const_name = [target.get_text() for target in const_name]
                    const_last = [re.sub('\n', ' ', target.get_text()) for target in const_last]
                    
                    if const_hipen == True: const_target = driver.find_element_by_id('constructor-summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-constructor-name')
                    else: const_target = driver.find_element_by_id('constructor.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-constructor-name')
                    const_target = [scol.find_elements_by_tag_name('a') for scol in const_target] #
                    
                    const_df = pd.DataFrame([const_first, const_name, const_last, const_target]).T
                    const = ['생성자', const_df] #######
                    
                    count += 1
                    const_sel = count
                    select_lists.append([count, const])  
            else: # 컬럼이 2개인 생성자 목록일 때
                const_cols = 2
                const_name = const_table.findAll('div', class_='col-constructor-name') # 이름 : 바로 생성자들부터 시작
                const_last = const_table.findAll('div', class_='col-last')[1:] # 설명 : 컬럼 이름부터 시작 : 그래서 1부터 슬라이싱

                const_name = [target.get_text() for target in const_name]
                const_last = [re.sub('\n', ' ', target.get_text()) for target in const_last]

                if const_hipen == True: const_target = driver.find_element_by_id('constructor-summary').find_elements_by_class_name('summary-table.two-column-summary')[0].find_elements_by_class_name('col-constructor-name')
                else: const_target = driver.find_element_by_id('constructor.summary').find_elements_by_class_name('summary-table.two-column-summary')[0].find_elements_by_class_name('col-constructor-name')
                const_target = [scol.find_elements_by_tag_name('a') for scol in const_target] #
                
                const_df = pd.DataFrame([const_name, const_last, const_target]).T
                const = ['생성자', const_df] #######

                count += 1
                const_sel = count
                select_lists.append([count, const])
                #print(const)

        # 메서드
        method_hipen = False
        method_raw = soup.find('section', id='method.summary')
        if method_raw == None: 
            method_hipen = True
            method_raw = soup.find('section', id='method-summary')
        if method_raw != None:
            method_title = method_raw.find('h2').get_text()
            method_table = method_raw.find('div', class_='summary-table three-column-summary')
            
            if method_table != None:
                method_col1 = method_table.findAll('div', class_='col-first')
                method_col2 = method_table.findAll('div', class_='col-second')
                method_col3 = method_table.findAll('div', class_='col-last')

                method_col1 = [target.get_text() for target in method_col1]
                method_col2 = [re.sub('\n', ' ', target.get_text()) for target in method_col2]
                method_col3 = [re.sub('\n', ' ', target.get_text()) for target in method_col3]
                
                if method_hipen == True: method_target = driver.find_element_by_id('method-summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                else: method_target = driver.find_element_by_id('method.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                method_target = [scol.find_elements_by_tag_name('a') for scol in method_target]
                
                method_df = pd.DataFrame([method_col1[1:], method_col2[1:], method_col3[1:], method_target[1:]]).T
                method = ['메서드', method_df] #######

                #밑의 방법으로 들어갈 수 있음.
                #method_target[1][0].click()
                count += 1
                select_lists.append([count, method])

        
    
    # 현재 페이지가 패키지인 경우
    elif category == 'Package': 
        swit = 2 # 대분류 두번째 표식
        titles = soup.findAll('div', class_='sub-title')
        
        # 모듈 : 패키지는 무조건 모듈밖에 없음!
        module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, module])
        
        # 본문
        content = soup.find('div', class_='block').get_text()
        
        # 인터페이스, 클래스, 열거형, 에러, 예외, 에너테이션 인터페이스를 모아놓은 summary-list
        summary_list = soup.find('ul', class_='summary-list')
        summary_lis = summary_list.findAll('li')
        midx = -1
        thcol = False
        for li in summary_lis:
            midx += 1
            # 분기할 기준. 제목을 가지고 종류를 분별.
            caption = li.find('div', class_='caption')
            if caption == None:
                caption = li.find('div', class_='table-tabs')
                if caption != None: caption = caption.find('button', id='class-summary-tab0').get_text()
            else: caption = caption.get_text()
            
            # 각각 종류별 목록들이 있는 테이블.
            table = li.find('div', class_='summary-table two-column-summary')
            if table != None:
                col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
                col_first = [col.get_text() for col in col_first]
                col_last = table.findAll('div', class_='col-last')[1:] # 두번째 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
                col_last = [col.get_text() for col in col_last]
            else:
                table = li.find('div', class_='summary-table three-column-summary')
                if table != None:
                    col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
                    col_first = [col.get_text() for col in col_first]
                    col_second = table.findAll('div', class_='col-second')[1:] # 두번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
                    col_second = [col.get_text() for col in col_second]
                    col_last = table.findAll('div', class_='col-last')[1:] # 두번째 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
                    col_last = [col.get_text() for col in col_last]
                    thcol = True
                    midx -= 1
                    
            #연관 패키지 목록. 예는 3칼럼이다.
            if caption == 'Related Packages':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.three-column-summary')
                if thcol == True: targets[midx].find_elements_by_class_name('col-second')[1:]
                else: targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                if thcol == True: rp_df = pd.DataFrame([col_first, col_second, col_last, col_targets]).T
                else: rp_df = pd.DataFrame([col_first, col_last, col_targets]).T
                rp = ['연관 패키지', rp_df]
                count += 1
                select_lists.append([count, rp])
            
            # 인터페이스 목록
            if caption == 'Interface Summary':
                #########################################################
                # 셀레니움으로 첫번째 열까지 찾는다. 여기에 링크가 있다. 마찬가지로, 첫번째는 그냥 칼럼 이름이기 때문에 슬라이싱 해준다.
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                # targets[원하는 항목 인덱스][0]에 click()나 send_keys(Keys.ENTER)을 쓰면 된다.
                ########################################################
                inter_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                inter = ['인터페이스', inter_df]
                count += 1
                select_lists.append([count, inter])
            
            # 클래스 목록
            elif caption == 'Class Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                classs_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                classs = ['클래스', classs_df]
                count += 1
                select_lists.append([count, classs])
                
            # 클래스와 인터페이스 목록
            elif caption == 'All Classes and Interfaces':
                print('idx : ', midx)
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                classinter_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                classinter = ['클래스와 인터페이스', classinter_df]
                count += 1
                select_lists.append([count, classinter])
            
            # 열거형 목록
            elif caption == 'Enum Class Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                enumc_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                enumc = ['열거형', enumc_df]
                count += 1
                select_lists.append([count, enumc])
            
            # 예외 목록
            elif caption == 'Exception Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                exc_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                exc = ['예외', exc_df]
                count += 1
                select_lists.append([count, exc])
                
            # 에러 목록
            elif caption == 'Error Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                errors_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                errors = ['에러', errors_df]
                count += 1
                select_lists.append([count, errors])
                
            # 애너테이션 인터페이스 목록
            elif caption == 'Annotation Interfaces Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                ais_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                ais = ['애너테이션 인터페이스', ais_df]
                count += 1
                select_lists.append([count, ais])
    
    # 현재 페이지가 모듈인 경우
    elif category == 'Module': 
        swit = 3 # 대분류 세번째 표식
        mod_tab = -1
        
        # 본문
        content = soup.find('div', class_='block').get_text()
        
        # 패키지 모음 : inderect Exports는 하위에 있응 모듈에서 들어가도 나오므로 Exports만 크롤링
        package_raw = soup.find('section', id='packages.summary')
        package_hipen = False
        if package_raw == None:
            package_raw = soup.find('section', id='packages-summary')
            package_hipen = True
        if package_raw != None:
            table = package_raw.find('div', class_='summary-table two-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_last = table.findAll('div', class_='col-last')[1:] # 두번째 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            if package_hipen == True: targets = driver.find_element_by_id('packages-summary').find_elements_by_class_name('summary-table.two-column-summary')[0].find_elements_by_class_name('col-first')[1:]
            else: targets = driver.find_element_by_id('packages.summary').find_elements_by_class_name('summary-table.two-column-summary')[0].find_elements_by_class_name('col-first')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            package_df = pd.DataFrame([col_first, col_last, col_targets]).T
            packages = ['패키지', package_df]
            count += 1
            select_lists.append([count, packages])
            
        # 모듈 모음 : 모듈은 컬럼이 3개이다.
        module_raw = soup.find('section', id='modules.summary')
        module_hipen = False
        if module_raw == None:
            module_raw = soup.find('section', id='modules-summary')
            module_hipen = True
        if module_raw != None:
            table = module_raw.find('div', class_='details-table three-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. modifier : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_second = table.findAll('div', class_='col-second')[1:] # 두번째 열. 이름 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_second = [col.get_text() for col in col_second]
            col_last = table.findAll('div', class_='col-last')[1:] # 마지막 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            if module_hipen == True: targets = driver.find_element_by_id('modules-summary').find_elements_by_class_name('details-table.three-column-summary')[0].find_elements_by_class_name('col-second')[1:]
            else: targets = driver.find_element_by_id('modules.summary').find_elements_by_class_name('details-table.three-column-summary')[0].find_elements_by_class_name('col-second')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            module_df = pd.DataFrame([col_first, col_second, col_last, col_targets]).T
            modules = ['모듈', module_df]
            count += 1
            mod_tab = count
            select_lists.append([count, modules])
            
        # 서비스 모음
        service_raw = soup.find('section', id='services.summary')
        service_hipen = False
        if service_raw == None:
            service_raw = soup.find('section', id='services-summary')
            service_hipen = True
        if service_raw != None:
            table = service_raw.find('div', class_='details-table two-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_last = table.findAll('div', class_='col-last')[1:] # 두번째 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            if service_hipen == True: targets = driver.find_element_by_id('services-summary').find_elements_by_class_name('details-table.two-column-summary')[0].find_elements_by_class_name('col-first')[1:]
            else: targets = driver.find_element_by_id('services.summary').find_elements_by_class_name('details-table.two-column-summary')[0].find_elements_by_class_name('col-first')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            service_df = pd.DataFrame([col_first, col_last, col_targets]).T
            services = ['서비스', service_df]
            count += 1
            select_lists.append([count, services])
    
    # 현재 페이지가 애너테이션 인터페이스인 경우 : 선택적 요소와 필수 요소로 나누인다.
    elif category == 'Annotation': 
        swit = 4 # 대분류 네번째 표식
        titles = soup.findAll('div', class_='sub-title')

        # 모듈
        module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, module])
        
        # 패키지
        package = ['패키지', titles[1].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[1].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, package])
        
        # 본문
        content = soup.find('div', class_='block').get_text()
        
        # optional element
        optional_element_raw = soup.find('section', id='annotation.interface.optional.element.summary')
        if optional_element_raw != None:
            print('optional')
            table = optional_element_raw.find('div', class_='summary-table three-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. modifier, 타입 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_second = table.findAll('div', class_='col-second')[1:] # 두번째 열. 이름 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_second = [col.get_text() for col in col_second]
            col_last = table.findAll('div', class_='col-last')[1:] # 마지막 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            targets = driver.find_element_by_id('annotation.interface.optional.element.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            oe_df = pd.DataFrame([col_first, col_second, col_last, col_targets]).T
            oes = ['선택적 요소', oe_df]
            count += 1
            select_lists.append([count, oes])
            
        # required element
        required_element_raw = soup.find('section', id='annotation.interface.required.element.summary')
        if required_element_raw != None:
            print('required')
            table = required_element_raw.find('div', class_='summary-table three-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. modifier, 타입 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_second = table.findAll('div', class_='col-second')[1:] # 두번째 열. 이름 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_second = [col.get_text() for col in col_second]
            col_last = table.findAll('div', class_='col-last')[1:] # 마지막 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            targets = driver.find_element_by_id('annotation.interface.required.element.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            req_df = pd.DataFrame([col_first, col_second, col_last, col_targets]).T
            req = ['필수 요소', req_df]
            count += 1
            select_lists.append([count, req])
    
    # 현재 페이지가 열거형 클래스인 경우
    elif category == 'Enum': 
        swit = 5 # 대분류 다섯번째 표식
        titles = soup.findAll('div', class_='sub-title')

        # 모듈
        module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, module])
        
        # 패키지
        package = ['패키지', titles[1].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[1].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, package])
        
         # 구현한 인터페이스
        section = soup.find('section', class_='description')
        cd = False
        if section == None: 
            section = soup.find('section', id='class-description')
            cd = True
        if section != None:
            notes = section.findAll('dl', class_='notes')
            idx = -1
            print('section in')
            for note in notes:
                idx += 1
                dt = note.find('dt')
                print('dt : ', dt.get_text())
                if dt.get_text() == 'All Implemented Interfaces:':
                    print('1')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['구현한 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                    
                elif dt.get_text() == 'All Superinterfaces:':
                    print('2')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['슈퍼 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                
                elif dt.get_text() == 'All Known Implementing Classes:':
                    print('3')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['구현 클래스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                
                elif dt.get_text() == 'All Known Subinterfaces:': 
                    print('4')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['서브 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                    
                elif dt.get_text() == 'Enclosing interface:': 
                    print('5')
                    imple_inter_title = note.find('dt').get_text()
                    imple_inters = note.select('dd > code > a')
                    imple_inters = [a.get_text() for a in imple_inters]
                    if cd == True: imple_targets = driver.find_element_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    imple_inter = ['Enclosing 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
                    count += 1
                    select_lists.append([count, imple_inter])
                    
                # 서브 클래스
                elif dt.get_text() == 'Direct Known Subclasses:':
                    print('6')
                    known_sub_title = note.find('dt').get_text()
                    known_subs = note.select('dd > code > a')
                    known_subs = [a.get_text() for a in known_subs]
                    if cd == True: known_targets = driver.find_elements_by_id('class-description').find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    else: known_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[idx].find_elements_by_tag_name('a')
                    known_sub = ['서브 클래스', [[name, target] for name, target in zip(known_subs, known_targets)]]
                    count += 1
                    select_lists.append([count, known_sub])
        
        # 본문
        content = soup.find('div', class_='block').get_text()
        
        # 중첩 클래스
        nested_raw = soup.find('section', id='nested.class.summary')
        if nested_raw != None:
            wrap = nested_raw.find('code')
            wrap = wrap.get_text() if wrap != None else '없음'
            target = driver.find_element_by_id('nested.class.summary').find_elements_by_tag_name('a')[0]
            nested = ['중첩 클래스', wrap, target]
            count += 1
            select_lists.append([count, nested])
            
        # 열거 상수
        two_row = -1
        constant_raw = soup.find('section', id='enum.constant.summary')
        if constant_raw != None:
            table = constant_raw.find('div', class_='summary-table two-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_last = table.findAll('div', class_='col-last')[1:] # 두번째 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            
            # 셀레니움으로 첫번째 열까지 찾는다. 여기에 링크가 있다. 마찬가지로, 첫번째는 그냥 칼럼 이름이기 때문에 슬라이싱 해준다.
            targets = driver.find_elements_by_id('enum.constant.summary')[0].find_elements_by_class_name('summary-table.two-column-summary')[0].find_elements_by_class_name('col-first')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            # targets[원하는 항목 인덱스][0]에 click()나 send_keys(Keys.ENTER)을 쓰면 된다.
            constant_df = pd.DataFrame([col_first, col_last, col_targets]).T
            constants = ['열거 상수', constant_df] #######
            count += 1
            two_row = count
            select_lists.append([count, constants])
            
        # 메서드
        method_raw = soup.find('section', id='method.summary')
        if method_raw != None:
            method_table = method_raw.find('div', class_='summary-table three-column-summary')
            
            if method_table != None:
                method_col1 = method_table.findAll('div', class_='col-first')
                method_col2 = method_table.findAll('div', class_='col-second')
                method_col3 = method_table.findAll('div', class_='col-last')

                method_col1 = [target.get_text() for target in method_col1]
                method_col2 = [re.sub('\n', ' ', target.get_text()) for target in method_col2]
                method_col3 = [re.sub('\n', ' ', target.get_text()) for target in method_col3]

                method_target = driver.find_element_by_id('method.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                method_target = [scol.find_elements_by_tag_name('a') for scol in method_target]
                
                method_df = pd.DataFrame([method_col1[1:], method_col2[1:], method_col3[1:], method_target[1:]]).T
                method = ['메서드', method_df] #######

                #밑의 방법으로 들어갈 수 있음.
                #method_target[1][0].click()
                count += 1
                select_lists.append([count, method])
                
    
    # 카카오톡으로 전송
    count += 1
    select_lists.append([count, ['카카오톡으로 전송']])
    
    # 본문 번역기 돌리기
    trans = Translator()
    
    if len(content) > 9000: content = summarize(content, ratio=0.5)
    at = trans.translate(content, src='en', dest='ko')
    if 100 < len(at.text) <= 300:
        summ = summarize(at.text, ratio=0.8)
    elif 300 < len(at.text) <= 700:
        summ = summarize(at.text, ratio=0.5)
    elif 700 < len(at.text):
        summ = summarize(at.text, ratio=0.2)
    else: summ = at.text
        
    # 사랑의 작대기
    memberin = False
    while True:
        if memberin == False:
            print()
            print(title)
            if swit in [1, 4, 5]: 
                sub_info = '({} : {} / {} : {})'.format(module[0], module[1], package[0], package[1])
                print(sub_info)
            elif swit == 2: 
                sub_info = '({} : {})'.format(module[0], module[1])
                print(sub_info)
            print()
            print(summ)
            print()
            print('메뉴')
            for count, sel in select_lists:
                print(count, '. ', sel[0], sep='', end='\t')
            sel = int(input('원하시는 메뉴를 선택해주세요. ====> '))
            
            
            if sel == count: return [7, title + '\n' + (sub_info if swit != 3 else '') + '\n' + summ]
            if select_lists[sel-1][1][0] in ['구현한 인터페이스', '슈퍼 인터페이스', '구현 클래스', '서브 인터페이스', 'Enclosing 인터페이스', '서브 클래스']:
                res = select_cat(select_lists[sel-1][1])
            elif swit == 1: # 대분류 첫번째이면 여기로
                # 무조건 method = ['메서드', method_df]가 통째로 들어간다. 필드, 메서드도 마찬가지!
                if const_sel != -1 and sel == const_sel: # 생성자가 있고, 생성자를 선택했다면
                    if const_cols == 2: # 근데, 생성자 목록 칼럼 수가 2개라면
                        res = select_cat(select_lists[sel-1][1], cols=2, trans=trans)
                elif select_lists[sel-1][1][0] in ['모듈', '패키지'] :
                    return [0, select_lists[sel-1][1]]
                else: # 어쨌건 생성자를 선택 안했다면
                    res = select_cat(select_lists[sel-1][1], trans=trans)
            elif swit == 2: # 대분류 두번째이면 여기로
                if select_lists[sel-1][1][0] == '연관 패키지' and thcol == True : res = select_cat(select_lists[sel-1][1], swit=3, trans=trans)
                elif select_lists[sel-1][1][0] == '모듈' : res = [5, select_lists[sel-1][1]]
                else: res = select_cat(select_lists[sel-1][1], cols=2, swit=2, trans=trans)
            elif swit == 3: # 대분류 세번째이면 여기로
                if sel == mod_tab: res = select_cat(select_lists[sel-1][1], swit=3, trans=trans)
                else: res = select_cat(select_lists[sel-1][1], cols=2, swit=2, trans=trans)
            elif swit == 4: # 대분류 네번째이면 여기로!
                res = select_cat(select_lists[sel-1][1], trans=trans)
            elif swit == 5:
                print('선택한 항목 : ', select_lists[sel-1][0])
                if select_lists[sel-1][1][0] in ['모듈', '패키지', '중첩 클래스']: return [0, select_lists[sel-1][1]]
                elif sel == two_row : res = select_cat(select_lists[sel-1][1], cols=2, trans=trans)
                else: res = select_cat(select_lists[sel-1][1], trans=trans)
            
        # res = [결과 코드, 결과 데이터(행)] 
        # 1. 구현 클래스, 구현한 인터페이스, 서브클래스
        # 2. 필드, 메서드, 생성자
        # 3. 중첩 클래스, 모듈 페이지 일부
        # 4. 패키지 페이지
        # 7. 카카오톡으로 전송(전송 후 종료)
        print('sel : {} ,  count : {}'.format(sel, count))
        if sel == count: return [7, title + '\n' + (sub_info if swit != 3 else '') + '\n' + summ]
        elif res[0] == 0: 
            if memberin == True: memberin = False
            continue
        elif res[0] == 1: return res
        elif res[0] == 2: 
            if len(res[1]) == 4: res[1][3][0].click()
            else: res[1][2][0].click()
            res = fd(driver)
            memberin = True
        elif res[0] == 3: return res
        elif res[0] == 4: return res
        elif res[0] == 5: return res
        elif res[0] == 7: return res
    
def select_cat(select_list, trans=None, cols=-1, swit=-1):
    count = 0
    print(select_list)
    if select_list[0] in ['구현한 인터페이스', '슈퍼 인터페이스', '구현 클래스', '서브 인터페이스', 'Enclosing 인터페이스', '서브 클래스']:
        for idx, li in enumerate(select_list[1]):
            print((idx+1), ' : ', li[0])
            count = idx+1
        print((count + 1), ' : ', '돌아가기')
        sel = int(input('2-0. 원하시는 메뉴를 선택해주세요. ====> '))
        if sel == count + 1: return [0, 0]
        else: return [1, select_list[1][sel-1]] # 구현 클래스, 구현한 인터페이스, 서브클래스이면 결과 코드를 1로
    # 필드, 생성자, 메서드
    elif cols == 2: # 칼럼이 2개일 때 (이건 거의 생성자 특수용. 필드나 메서드는 대부분 칼럼이 3개이다.)
        for i in range(len(select_list[1])):
            count += 1
            row = select_list[1].iloc[i]
            # trans.translate(row[1], src='en', dest='ko').text
            #print((count), ' : ', row[0], row[1], '\n')
            print((count), ' : ', row[0], row[1], '\n')
        print((count + 1), ' : ', '돌아가기')
        sel = int(input('2. 원하시는 메뉴를 선택해주세요. ====> ')) 
        if sel == count + 1: return [0, 0]
        else:
            if swit == 2: return [4, select_list[1].iloc[sel-1]] # 패키지 페이지이면 결과 코드를 4로
            else: return [2, select_list[1].iloc[sel-1]]
    else: # 칼럼이 3개!
        for i in range(len(select_list[1])):
            count += 1
            
            row = select_list[1].iloc[i]
            #print((count), ' : ', row[0], row[1], '\n', row[2], '\n')
            print((count), ' : ', row[0], row[1], '\n', row[2], '\n')
        print((count + 1), ' : ', '돌아가기')
        sel = int(input('2. 원하시는 메뉴를 선택해주세요. ====> ')) 
        if sel == count + 1: return [0, 0]
        else: 
            if select_list[0] == '중첩 클래스' or swit == 3: return [3, select_list[1].iloc[sel-1]] # 중첩 클래스이거나 모듈 페이지이면 결과 코드를 3으로
            else: return [2, select_list[1].iloc[sel-1]] # 필드, 메서드, 생성자이면 결과 코드를 2로
    
# 메인 로직
driver = execute()
time.sleep(3)

#kill_iframe(driver)
result = ''
url = driver.current_url
is_member = False
if '#' in url: is_member = True
while True:
    if is_member == True:
        result = fd(driver)
    else: result = main_process(driver)
    
    if is_member == True and result[0] == 0:
        is_member = False
        result = main_process(driver)
    if result[0] == 0: result[1][2].send_keys(Keys.ENTER)
    elif result[0] == 1: result[1][1].send_keys(Keys.ENTER)
    elif result[0] == 2: # 필드, 메서드, 생성자, 
        if len(result[1]) == 4: result[1][3][0].click()
        else: result[1][2][0].click()
        break
    elif result[0] == 3: # 중첩 클래스일 때
        result[1][3][0].click()
    elif result[0] == 4: # 패키지 페이지에서 선택했을 때, 즉, swit == 2일 때
        result[1][2][0].click()
    elif result[0] == 5:
        result[1][2].send_keys(Keys.ENTER)
    elif result[0] == 7:
        kakaoTalk = True
        break
        
if kakaoTalk == True:
    print()
    print(result[1])
    print()
    send_message(result[1])
    print('='*30, 'The End', '='*30)

입력창입니다. 입력하세요.primitiveiterator
Category :  Classes and Interfaces
1 :  java.util.PrimitiveIterator
원하시는 번호를 입력해주세요.1


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x0116B8F3+2406643]
	Ordinal0 [0x010FAF31+1945393]
	Ordinal0 [0x00FEC748+837448]
	Ordinal0 [0x00FEF154+848212]
	Ordinal0 [0x00FEF012+847890]
	Ordinal0 [0x00FEF2A0+848544]
	Ordinal0 [0x0101ACB9+1027257]
	Ordinal0 [0x01011493+988307]
	Ordinal0 [0x0103427C+1131132]
	Ordinal0 [0x0100FA64+981604]
	Ordinal0 [0x01034494+1131668]
	Ordinal0 [0x01044682+1197698]
	Ordinal0 [0x01034096+1130646]
	Ordinal0 [0x0100E636+976438]
	Ordinal0 [0x0100F546+980294]
	GetHandleVerifier [0x013D9612+2498066]
	GetHandleVerifier [0x013CC920+2445600]
	GetHandleVerifier [0x01204F2A+579370]
	GetHandleVerifier [0x01203D36+574774]
	Ordinal0 [0x01101C0B+1973259]
	Ordinal0 [0x01106688+1992328]
	Ordinal0 [0x01106775+1992565]
	Ordinal0 [0x0110F8D1+2029777]
	BaseThreadInitThunk [0x76AFFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77727A9E+286